<a href="https://colab.research.google.com/github/yasu-k2/multimodal-active-inference/blob/main/AudioVisualMaze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pygame
!pip install inferactively-pymdp

     |████████████████████████████████| 21.8 MB 3.0 MB/s 
     |████████████████████████████████| 60 kB 3.2 MB/s 
     |████████████████████████████████| 154 kB 10.4 MB/s 
     |████████████████████████████████| 4.3 MB 62.9 MB/s 
     |████████████████████████████████| 297 kB 65.1 MB/s 
     |████████████████████████████████| 38.1 MB 330 kB/s 
     |████████████████████████████████| 149 kB 50.0 MB/s 
     |████████████████████████████████| 2.8 MB 36.4 MB/s 
     |████████████████████████████████| 41 kB 27 kB/s 
     |████████████████████████████████| 46 kB 2.4 MB/s 
     |████████████████████████████████| 3.1 MB 42.5 MB/s 
     |████████████████████████████████| 5.3 MB 56.9 MB/s 
     |████████████████████████████████| 70 kB 6.9 MB/s 
     |████████████████████████████████| 41 kB 23 kB/s 
     |████████████████████████████████| 83 kB 1.8 MB/s 
     |████████████████████████████████| 130 kB 55.2 MB/s 
     |████████████████████████████████| 428 kB 46.6 MB/s 
     |██████████████████████

In [2]:
!git clone https://github.com/yasu-k2/multimodal-active-inference.git

Cloning into 'multimodal-active-inference'...
remote: Enumerating objects: 143, done.
remote: Counting objects: 100% (143/143), done.
remote: Compressing objects: 100% (114/114), done.
remote: Total 143 (delta 56), reused 58 (delta 9), pack-reused 0
Receiving objects: 100% (143/143), 2.88 MiB | 12.82 MiB/s, done.
Resolving deltas: 100% (56/56), done.


In [3]:
%cd multimodal-active-inference

/content/multimodal-active-inference


In [4]:
!git pull origin main

From https://github.com/yasu-k2/multimodal-active-inference
 * branch            main       -> FETCH_HEAD
Already up to date.


## Audio Visual Maze Setup

In [16]:
import numpy as np

from AudioVisualMazeEnv import print_maze, fn_create_maze, create_maze, AudioVisualMazeEnv

In [6]:
%load_ext autoreload
%autoreload 2

In [12]:
maze_dir = './'
n_div_sound = 7
maze_height = 11
maze_width = 11
create = True
seed = 0
env = AudioVisualMazeEnv(n_div_sound=n_div_sound, maze_height=maze_height, maze_width=maze_width,
                         create=create, seed=seed, start_pos=(1,1), end_pos='LowerRight', DIR=maze_dir)

start to create 11 x 11 maze
finish
Starting state is (1, 1)


In [13]:
actions = ["UP", "DOWN", "LEFT", "RIGHT", "STAY"]
n_actions = len(actions)

In [17]:
T = 3
obs = env.reset()
print('obs :', obs[1])
print_maze(obs[0])
for t in range(T):
  action_index = np.random.choice(len(actions))
  # action_index = 1
  action_label = actions[action_index]
  obs, rewards, done, info = env.step(action_label)
  print('=====Time {}====='.format(t+1))
  env.render()
  print(env.current_state)  # [Y, X, audio]
  print('obs :', obs[1])
  print_maze(obs[0])

Re-initialized location to (1, 1, 6)
■■■■■■■■■■■
■◎　　■　　　　　■
■■■　■　■　■　■
■　■　■　■　■　■
■　■　■　■　■■■
■　■　■　■　　　■
■　■　■　■■■　■
■　■　■　　　■　■
■　■　■■■■■　■
■　　　　　　　　★■
■■■■■■■■■■■
..and sampled observation [array([[1., 1., 1.],
       [1., 2., 0.],
       [1., 1., 1.]]), 6]
obs : 6
■■■
■◎　
■■■
=====Time 1=====
■■■■■■■■■■■
■◎　　■　　　　　■
■■■　■　■　■　■
■　■　■　■　■　■
■　■　■　■　■■■
■　■　■　■　　　■
■　■　■　■■■　■
■　■　■　　　■　■
■　■　■■■■■　■
■　　　　　　　　★■
■■■■■■■■■■■
[1, 1, 6]
obs : 6
■■■
■◎　
■■■
=====Time 2=====
■■■■■■■■■■■
■◎　　■　　　　　■
■■■　■　■　■　■
■　■　■　■　■　■
■　■　■　■　■■■
■　■　■　■　　　■
■　■　■　■■■　■
■　■　■　　　■　■
■　■　■■■■■　■
■　　　　　　　　★■
■■■■■■■■■■■
[1, 1, 6]
obs : 6
■■■
■◎　
■■■
=====Time 3=====
■■■■■■■■■■■
■　◎　■　　　　　■
■■■　■　■　■　■
■　■　■　■　■　■
■　■　■　■　■■■
■　■　■　■　　　■
■　■　■　■■■　■
■　■　■　　　■　■
■　■　■■■■■　■
■　　　　　　　　★■
■■■■■■■■■■■
[1, 2, 6]
obs : 6
■■■
◎　　
■■　


## Active Inference Agent

In [18]:
import copy
import itertools

# import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pymdp
from pymdp import utils
from pymdp.agent import Agent
from pymdp.maths import softmax

In [19]:
def plot_grid(grid_locations, num_y=21, num_x=21):
  grid_heatmap = np.zeros((num_y, num_x))
  for linear_idx, location in enumerate(grid_locations):
    y, x = location
    grid_heatmap[y, x] = linear_idx
  sns.heatmap(grid_heatmap, annot=True, cbar=False, fmt='.0f', cmap='crest')

def plot_point_on_grid(state_vector, grid_locations, num_y=21, num_x=21):
  state_index = np.where(state_vector)[0][0]
  print(np.where(state_vector))
  y, x = grid_locations[state_index]
  grid_heatmap = np.zeros((num_y, num_x))
  grid_heatmap[y,x] = 1.0
  sns.heatmap(grid_heatmap, cbar=False, fmt='.0f')

def plot_likelihood(matrix, title_str=""):
  if not np.isclose(matrix.sum(axis=0), 1.0).all():
    raise ValueError("Distrbution not column-normalized.")
  sns.heatmap(matrix, cmap='OrRd', vmin=0.0, vmax=1.0)
  plt.xticks(range(A.shape[1]))
  plt.yticks(range(A.shape[0]))
  plt.title(title_str)

def plot_beliefs(belief_dist, title_str=""):
  if not np.isclose(belief_dist.sum(), 1.0):
    raise ValueError("Distribution not normalized.")
  plt.grid(zorder=0)
  plt.bar(range(belief_dist.shape[0], belief_dist, color='r', zorder=3))
  plt.xticks(range(belief_dist.shape[0]))
  plt.title(title_str)

In [20]:
def create_B_matrix(maze_array, grid_locations, actions):
  maze_height, maze_width = maze_array.shape
  n_states = len(grid_locations)
  n_action = len(actions)
  B = np.zeros((n_states, n_states, n_actions))
  for action_id, action_label in enumerate(actions):
    for curr_state, grid_location in enumerate(grid_locations):
      y, x = grid_location
      if action_label == "UP":
        next_y = y - 1 if y > 0 else y
        next_x = x
      elif action_label == "DOWN":
        next_y = y + 1 if y < (maze_height-1) else y
        next_x = x
      elif action_label == "LEFT":
        next_x = x - 1 if x > 0 else x
        next_y = y
      elif action_label == "RIGHT":
        next_x = x + 1 if x < (maze_width-1) else x
        next_y = y
      elif action_label == "STAY":
        next_x = x
        next_y = y

      # Blocked by wall
      if maze_array[next_y, next_x] == 1:
        next_x = x
        next_y = y

      next_location = (next_y, next_x)
      next_state = grid_locations.index(next_location)
      B[next_state, curr_state, action_id] = 1.0
  return B

In [21]:
grid_locations = list(itertools.product(range(maze_height), range(maze_width)))
# print(len(grid_locations), grid_locations)
# plot_grid(grid_locations)

In [22]:
n_location_states = len(grid_locations)  # current position
n_maze_states = len(grid_locations)  # blocked or non-blocked
n_states = [n_location_states]  # + [2] * n_maze_states
n_factors = len(n_states)
n_controls = [n_actions]  # + [1] * n_maze_states

n_location_observations = len(grid_locations)  # current position
n_sound_observations = n_div_sound  # sound cue intensity
n_reward_observations = 2  # no reward or reward
n_obs = [n_location_observations, n_sound_observations, n_reward_observations]
n_modalities = len(n_obs)

In [25]:
# Generate the A array
A = utils.obj_array(n_modalities)

# first consider a known maze state
A_location = np.zeros((n_location_observations, n_location_states))
A_location[:, :] = np.eye(n_location_states)
# print(utils.is_normalized(A_location))
A[0] = A_location

A_sound = np.zeros((n_sound_observations, n_location_states))
# print(env.bins)
for gli, gl in enumerate(grid_locations):
  sli = env.compute_sound(gl[0], gl[1])
  A_sound[sli, gli] = 1.0
# print(utils.is_normalized(A_sound))
A[1] = A_sound

A_reward = np.zeros((n_reward_observations, n_location_states))
goal_location = (maze_height-2, maze_width-2)
goal_location_index = grid_locations.index(goal_location)
A_reward[0, :] = 1.0
A_reward[0, goal_location_index] = 0.0
A_reward[1, goal_location_index] = 1.0
# print(utils.is_normalized(A_reward))
A[2] = A_reward

In [26]:
# Generate the B array
B = utils.obj_array(n_factors)
maze_array = copy.deepcopy(env.maze_array)
B_location = create_B_matrix(maze_array, grid_locations, actions)
# print(utils.is_normalized(B_location))
B[0] = B_location

B_maze = np.zeros((2, 2, 1))
B_maze[:,:,0] = np.eye(2)
# print(utils.is_normalized(B_maze))
for idx in range(1, n_factors):
  B[idx] = B_maze

In [27]:
# Generate the C array
C = utils.obj_array_zeros(n_obs)

C_location = np.zeros(n_location_observations)
desired_location = (maze_height-2, maze_width-2)  # lower right
desired_location_index = grid_locations.index(desired_location)
C_location[desired_location_index] = 1.0
# print(utils.is_normalized(C_location))

C_sound = np.zeros(n_sound_observations)
desired_sound_index = 0  # smallest sound
C_sound[desired_sound_index] = 1.0
# print(utils.is_normalized(C_sound))

C_reward = np.zeros(n_reward_observations)
C_reward[1] = 1.0
# print(utils.is_normalized(C_reward))

# Choose your preference
# C[0] = C_location
# C[1] = C_sound
C[2] = C_reward

In [28]:
# Generate the D array
D = utils.obj_array(n_factors)

starting_location = (1, 1)
starting_state_index = grid_locations.index(starting_location)
starting_state = utils.onehot(starting_state_index, n_location_states)
# plot_point_on_grid(starting_state, grid_locations)
D_location = utils.onehot(starting_state_index, n_location_states)
# print(utils.is_normalized(D_location))
D[0] = D_location

D_maze = np.array([0.5, 0.5])
# print(utils.is_normalized(D_maze))
for idx in range(1, n_factors):
  D[idx] = D_maze

In [29]:
def run_active_inference_loop(my_agent, my_env, T=3):
  obs = env.reset()
  reward = 0
  for t in range(T):
    env.render()
    print('obs :', obs[1])
    print_maze(obs[0])
  
    location = (env.current_state[0], env.current_state[1])
    sound = obs[1]
    obs_agent = [grid_locations.index(location), sound, reward]
    qs = my_agent.infer_states(obs_agent)
    q_pi, efe = my_agent.infer_policies()
    action_id = my_agent.sample_action()
    action_index = int(action_id[0])

    action_label = actions[action_index]
    obs, reward, done, info = env.step(action_label)
    print('=====Time {}====='.format(t+1))

    print(f'Action at time {t+1}: {action_label}')
    print(f'Reward at time {t+1}: {reward}')

In [30]:
policy_len = 5
controllable_indices = [0]
action_selection = 'deterministic'  # 'deterministic', 'stochastic'
my_agent = Agent(A=A, B=B, C=C, D=D,
                 policy_len=policy_len,
                 control_fac_idx=controllable_indices,
                 action_selection=action_selection)

In [31]:
T = 3
run_active_inference_loop(my_agent, env, T=T)

Re-initialized location to (1, 1, 6)
■■■■■■■■■■■
■◎　　■　　　　　■
■■■　■　■　■　■
■　■　■　■　■　■
■　■　■　■　■■■
■　■　■　■　　　■
■　■　■　■■■　■
■　■　■　　　■　■
■　■　■■■■■　■
■　　　　　　　　★■
■■■■■■■■■■■
..and sampled observation [array([[1., 1., 1.],
       [1., 2., 0.],
       [1., 1., 1.]]), 6]
■■■■■■■■■■■
■◎　　■　　　　　■
■■■　■　■　■　■
■　■　■　■　■　■
■　■　■　■　■■■
■　■　■　■　　　■
■　■　■　■■■　■
■　■　■　　　■　■
■　■　■■■■■　■
■　　　　　　　　★■
■■■■■■■■■■■
obs : 6
■■■
■◎　
■■■
=====Time 1=====
Action at time 1: UP
Reward at time 1: 0
■■■■■■■■■■■
■◎　　■　　　　　■
■■■　■　■　■　■
■　■　■　■　■　■
■　■　■　■　■■■
■　■　■　■　　　■
■　■　■　■■■　■
■　■　■　　　■　■
■　■　■■■■■　■
■　　　　　　　　★■
■■■■■■■■■■■
obs : 6
■■■
■◎　
■■■
=====Time 2=====
Action at time 2: UP
Reward at time 2: 0
■■■■■■■■■■■
■◎　　■　　　　　■
■■■　■　■　■　■
■　■　■　■　■　■
■　■　■　■　■■■
■　■　■　■　　　■
■　■　■　■■■　■
■　■　■　　　■　■
■　■　■■■■■　■
■　　　　　　　　★■
■■■■■■■■■■■
obs : 6
■■■
■◎　
■■■
=====Time 3=====
Action at time 3: UP
Reward at time 3: 0


## ActInf with learning

Choose the matrix you want to learn. Learning A in the example below.

In [92]:
# Agent(A=A, pA=pA, modalities_to_learn=learnable_modalities, lr_pA=1.0)
#  agent.update_A(obs): update_obs_likelihood_dirichlet(pA, A, obs, qs)
pA = utils.dirichlet_like(A, scale=1.0)
# TODO: do something with pA
A_gm = utils.norm_dist_obj_arr(pA)

# Agent(B=B, pB=pB, factors_to_learn=learnable_factors, lr_pB=1.0)
#   agent.update_B(qs_prev): update_state_likelihood_dirichlet(pB, B, actions, qs, qs_prev)
pB = utils.dirichlet_like(B, scale=1.0)
# TODO: do something with pB
B_gm = utils.norm_dist_obj_arr(pB)

# Agent(D=D, pD=pD, factors_to_learn=learnable_factors, lr_pD=1.0)
#   agent.update_D(qs_t0=None): update_state_prior_dirichlet(pD, qs)
pD = utils.dirichlet_like(D, scale=1.0)
# TODO: do something with pD
D_gm = utils.norm_dist_obj_arr(pD)

learnable_modalities = [0, 1]  # TODO: specify indices
# learnable_factors = [0]  # TODO: specify indices
my_agent = Agent(A=A_gm, B=B, C=C, D=D,
                 pA=pA, lr_pA=1.0,
                 policy_len=policy_len,
                 control_fac_idx=controllable_indices,
                 action_selection=action_selection,
                 modalities_to_learn=learnable_modalities,
                 use_param_info_gain=True)

In [93]:
def run_active_inference_loop_with_learning(my_agent, my_env, T=3):
  obs = env.reset()
  reward = 0
  for t in range(T):
    # env.render()
    location = (env.current_state[0], env.current_state[1])
    sound = obs[1]
    obs_agent = [grid_locations.index(location), sound, reward]
    qs = my_agent.infer_states(obs_agent)
    q_pi, efe = my_agent.infer_policies()
    action_id = my_agent.sample_action()
    pA_t = my_agent.update_A(obs_agent)  # update A via pA

    action_index = int(action_id[0])
    action_label = actions[action_index]
    obs, reward, done, info = env.step(action_label)    

In [94]:
T = 3
run_active_inference_loop_with_learning(my_agent, env, T=T)

Re-initialized location to (1, 1, 10)
■■■■■■■■■■■■■■■■■■■■■
■　■　　　　　　　　　　　　　■　　　■
■　■■■■■　■■■■■■■　■　■　■
■　■　　　■　　　　　■　　　　　■　■
■　■　■　■　■■■　■■■■■■■　■
■　　　■　■　　　■　　　■　■　　　■
■■■■■　■■■■■■■　■　■　■■■
■　　　■　　　　　　　■　■　■　■　■
■　■　■■■■■■■　■　■　■　■　■
■　■　　　■　　　■　■　■　　　　　■
■　■■■　■　■　■　■　■■■■■■■
■　■　■　■　■　　　■　　　　　　　■
■　■　■　■　■■■■■■■■■■■　■
■　■　　　■　　　　　■　　　　　　　■
■　■　■■■■■■■　■　■■■■■　■
■　■　　　■　　　■　　　■　　　　　■
■　■■■　■　■■■■■■■　■■■■■
■　■　　　　　■　　　　　■　　　　　■
■　■■■■■■■　■■■　■■■■■　■
■　　　　　　　　　　　■　　　　　　★■
■■■■■■■■■■■■■■■■■■■■■
..and sampled observation [[1. 1. 1.]
 [1. 0. 1.]
 [1. 0. 1.]]
■■■■■■■■■■■■■■■■■■■■■
■　■　　　　　　　　　　　　　■　　　■
■◎■■■■■　■■■■■■■　■　■　■
■　■　　　■　　　　　■　　　　　■　■
■　■　■　■　■■■　■■■■■■■　■
■　　　■　■　　　■　　　■　■　　　■
■■■■■　■■■■■■■　■　■　■■■
■　　　■　　　　　　　■　■　■　■　■
■　■　■■■■■■■　■　■　■　■　■
■　■　　　■　　　■　■　■　　　　　■
■　■■■　■　■　■　■　■■■■■■■
■　■　■　■　■　　　■　　　　　　　■
■　■　■　■　■■■■■■■■■■■　■
■　■　　　■　　　　　■　　　　　　　■
■　■　■■■■■■■　■　■■■■■　■
■　■　　　■　　　■　　　■　　　　　■
■　■■■　■　■■■■■■■　■■■■■
■　■　　　　　■　　　　　■　　　　　■
■　■■■■■■■　■■■　■■■■■　■
■　　　　　　　　　　　■　　　　　　